In [92]:
import pandas as pd
import numpy as np

from keras.layers import *
from keras.models import *
from keras.optimizers import *
from keras.applications import *
from keras.preprocessing.image import ImageDataGenerator

In [93]:
# Parameter setting
np.random.seed(2018)
model_name = 'Xception'
img_size = (299, 299)
batch_size = 16
freeze_layer_num = 20

In [94]:
# Data preparation
train_generator = ImageDataGenerator(preprocessing_function = xception.preprocess_input,
                                        rotation_range=10*2,
                                        width_shift_range=0.05*2,
                                        height_shift_range=0.05*2,
                                        shear_range=0.1*2,
                                        zoom_range=0.1*2,)
valid_test_gen = ImageDataGenerator(preprocessing_function = xception.preprocess_input)

train_data_generator = train_generator.flow_from_directory('.\\imgs\\new_train',
                                                          target_size=img_size,
                                                          batch_size=batch_size,
                                                          shuffle=True,
                                                          class_mode='categorical')
valid_data_generator = valid_test_gen.flow_from_directory('.\\imgs\\new_valid',
                                                          target_size=img_size,
                                                          batch_size=batch_size,
                                                          shuffle=True,
                                                          class_mode='categorical')
test_data_generator = valid_test_gen.flow_from_directory('.\\imgs\\test',
                                                          target_size=img_size,
                                                          batch_size=batch_size,
                                                          shuffle=False,
                                                          class_mode=None)

Found 20271 images belonging to 10 classes.
Found 2153 images belonging to 10 classes.
Found 79726 images belonging to 1 classes.


In [95]:
# Construct model
from keras import backend as K
K.clear_session()

inputs = Input((*img_size, 3))
base_model = Xception(input_tensor=inputs, weights='imagenet', include_top=False)
x = GlobalAveragePooling2D()(base_model.output)
x = Dropout(0.5)(x)
x = Dense(10, activation='softmax')(x)
model = Model(inputs, x)

print("total layer count {}".format(len(base_model.layers)))

total layer count 132


In [96]:
# Train the FC layer
base_model.trainabel = False
model.compile(optimizer=Adam(lr=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(train_data_generator,
                    steps_per_epoch=len(train_data_generator)*0.2,
                    epochs=1,
                    validation_data=valid_data_generator,
                    validation_steps=len(valid_data_generator))

Epoch 1/1
254/253 [==============================] - 189s 743ms/step - loss: 0.9014 - acc: 0.7210 - val_loss: 0.3518 - val_acc: 0.8927


In [97]:
# fine-tune the model
for i, layer in enumerate(model.layers):
    if i <= freeze_layer_num:
        layer.trainable = False
    else:
        layer.trainable = True
    
model.compile(optimizer=RMSprop(lr=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit_generator(train_data_generator, 
                    steps_per_epoch=len(train_data_generator)*0.2,
                    epochs=1,
                    validation_data=valid_data_generator, 
                    validation_steps=len(valid_data_generator))

Epoch 1/1
254/253 [==============================] - 157s 617ms/step - loss: 0.1360 - acc: 0.9606 - val_loss: 0.3761 - val_acc: 0.9048


In [99]:
# Train model
model.compile(optimizer=Adam(lr=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit_generator(train_data_generator, 
                    steps_per_epoch=len(train_data_generator),
                    epochs=1,
                    validation_data=valid_data_generator, 
                    validation_steps=len(valid_data_generator))

Epoch 1/1
1267/1267 [==============================] - 959s 757ms/step - loss: 0.0661 - acc: 0.9810 - val_loss: 0.2020 - val_acc: 0.9340


In [101]:
if not os.path.exists('.\\models'):
    os.mkdir('.\\models')
model.save('models/'+model_name+'_L'+ str(freeze_layer_num)+'.h')

In [102]:
# Train model
model.compile(optimizer=Adam(lr=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit_generator(train_data_generator, 
                    steps_per_epoch=len(train_data_generator)*0.2,
                    epochs=1,
                    validation_data=valid_data_generator, 
                    validation_steps=len(valid_data_generator))

Epoch 1/1
254/253 [==============================] - 160s 630ms/step - loss: 0.0253 - acc: 0.9934 - val_loss: 0.2349 - val_acc: 0.9378


In [103]:
# submition file
#model = load_model('models/'+model_name+'_L'+ str(freeze_layer_num)+'.h')
df = pd.read_csv("submit_example.csv", index_col='img')
preds = model.predict_generator(test_data_generator, steps=len(test_data_generator), verbose=1)
for i, fname in enumerate(test_data_generator.filenames):
    index = fname[fname.rfind('\\')+1:]
    df.loc[index] = preds[i]

df.to_csv('submission/'+model_name+'_L'+ str(freeze_layer_num)+'pred.csv', float_format='%.3f')

4983/4983 [==============================] - 1019s 205ms/step
